<table>
  <tr style="background-color: transparent;">
    <td width=20% align="center"><img width="100" src="https://www.ingenieria.unam.mx/nuestra_facultad/images/institucionales/escudos/escudounam_negro.jpg"/></td>
    <td width=60%><font size="7">Post-Quantum Cryptography Project</font></td>
    <td width=20% align="center"><img width="100" src="https://www.ingenieria.unam.mx/nuestra_facultad/images/institucionales/escudos/escudofi_negro.jpg"/></td>
  </tr>
  <tr style="background-color: transparent; text-align:center;">
    <td></td>
    <td align="center"><font size="3">2nd Project: Cryptography</font></td>
    <td></td>
  </tr>
</table>

**Students:**
* Andres Urbano Andrea
* Aguilar Corona Fernanda
* Barrios López Francisco
* Castillo Montes Pamela
* Ramírez Gómez María Emilia


<table>
  <tr style="background-color: transparent;">
    <td width=70%></td>
    <td width=10%><font size="3"><strong>Date: </strong>30/05/2024</font></td>
  </tr>
</table>

## ML-KEM Scheme

In [5]:
import oqs
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad
from timeit import default_timer

def ML_KEM(plain_text: str, ML_KEM_alg: str) -> dict:
    """Function that uses the ML-KEM scheme and encrypt and decrypt a message"""
    with oqs.KeyEncapsulation(ML_KEM_alg) as client:
        with oqs.KeyEncapsulation(ML_KEM_alg) as server:
            # Dictionary to save timestamps
            ML_KEM_time = {
                "key_pair_generation": [],
                "encap_secret": [],
                "decap_secret": []
                }

            ML_KEM_time["key_pair_generation"].append(default_timer())
            # Client generates its keypair, in this case, he only gets his public key
            # but he can get his provate key using export_secret_key()
            public_key_client = client.generate_keypair()
            ML_KEM_time["key_pair_generation"].append(default_timer())

            ML_KEM_time["encap_secret"].append(default_timer())
            # The server encapsulates its secret using the client's public key
            ciphertext, shared_secret_server = server.encap_secret(public_key_client)
            ML_KEM_time["encap_secret"].append(default_timer())

            ML_KEM_time["decap_secret"].append(default_timer())
            # The client decapsulates the server's ciphertext to obtain the shared secret
            shared_secret_client = client.decap_secret(ciphertext)
            ML_KEM_time["decap_secret"].append(default_timer())

            # We compare in both shared secret keys are equal
            if shared_secret_client == shared_secret_server:
                msg=plain_text.encode('utf-8')
                msgPad=pad(msg, 256)

                #Cifrado Simétrico con la llave generada
                AES_ECBcypher=AES.new(shared_secret_client, AES.MODE_ECB)

                ciphertext_ld = AES_ECBcypher.encrypt(msgPad)
                descifrado_aes_ld = AES_ECBcypher.decrypt(ciphertext_ld)

            return ML_KEM_time

average_time = {
    "key_pair_generation_ML-KEM-512": [],
    "encap_secret_ML-KEM-512": [],
    "decap_secret_ML-KEM-512": [],
    "total_ML-KEM-512": [],
    "key_pair_generation_ML-KEM-768": [],
    "encap_secret_ML-KEM-768": [],
    "decap_secret_ML-KEM-768": [],
    "total_ML-KEM-768": [],
    "key_pair_generation_ML-KEM-1024": [],
    "encap_secret_ML-KEM-1024": [],
    "decap_secret_ML-KEM-1024": [],
    "total_ML-KEM-1024": [],
    }

# We repeat each algorithm a hundred times
for _ in range(1000):
    for algorithm in ["ML-KEM-512", "ML-KEM-768", "ML-KEM-1024"]:
        times = ML_KEM("hello world", algorithm)
        #print(times)
        total_time = 0
        for key, value in times.items():
            #print(key)
            #print(value)
            total_time += value[1] - value[0]
            average_time[key+"_"+algorithm].append(value[1] - value[0])
            #print(average_time[key+"_"+algorithm])
        average_time["total_"+algorithm].append(total_time)

# We show the final results
for key, value in average_time.items():
    if "key_pair_generation" in key: print(f"\t ### {key[20:]} ###")
    print(f"{key}")
    print(sum(value))


	 ### ML-KEM-512 ###
key_pair_generation_ML-KEM-512
0.33341809996636584
encap_secret_ML-KEM-512
0.33271489906474017
decap_secret_ML-KEM-512
0.2859877008595504
total_ML-KEM-512
0.9521206998906564
	 ### ML-KEM-768 ###
key_pair_generation_ML-KEM-768
0.42969050008105114
encap_secret_ML-KEM-768
0.4796692001691554
decap_secret_ML-KEM-768
0.4648727996973321
total_ML-KEM-768
1.3742324999475386
	 ### ML-KEM-1024 ###
key_pair_generation_ML-KEM-1024
0.575093999854289
encap_secret_ML-KEM-1024
0.6532084998325445
decap_secret_ML-KEM-1024
0.659086399973603
total_ML-KEM-1024
1.8873888996604364


## ML-DSA Scheme

In [3]:
import oqs
from pprint import pprint

print("Enabled key encapsulation mechanisms:")
key_encap_meca = oqs.get_enabled_kem_mechanisms()
pprint(key_encap_meca, compact=True)

print("Enabled signature mechanisms:")
sigs = oqs.get_enabled_sig_mechanisms()
pprint(sigs, compact=True)

message = "This is the message to sign".encode()

# Create signer and verifier with sample signature mechanisms
sigalg = "Dilithium2"
with oqs.Signature(sigalg) as signer:
    with oqs.Signature(sigalg) as verifier:
        print("\nSignature details:")
        pprint(signer.details)

        # Signer generates its keypair
        signer_public_key = signer.generate_keypair()
        # Optionally, the secret key can be obtained by calling export_secret_key()
        # and the signer can later be re-instantiated with the key pair:
        # secret_key = signer.export_secret_key()

        # Store key pair, wait... (session resumption):
        # signer = oqs.Signature(sigalg, secret_key)

        # Signer signs the message
        signature = signer.sign(message)

        # Verifier verifies the signature
        is_valid = verifier.verify(message, signature, signer_public_key)

        print("\nValid signature?", is_valid)

Enabled key encapsulation mechanisms:
['Classic-McEliece-348864', 'Classic-McEliece-348864f',
 'Classic-McEliece-460896', 'Classic-McEliece-460896f',
 'Classic-McEliece-6688128', 'Classic-McEliece-6688128f',
 'Classic-McEliece-6960119', 'Classic-McEliece-6960119f',
 'Classic-McEliece-8192128', 'Classic-McEliece-8192128f', 'HQC-128', 'HQC-192',
 'HQC-256', 'Kyber512', 'Kyber768', 'Kyber1024', 'ML-KEM-512-ipd', 'ML-KEM-512',
 'ML-KEM-768-ipd', 'ML-KEM-768', 'ML-KEM-1024-ipd', 'ML-KEM-1024', 'sntrup761',
 'FrodoKEM-640-AES', 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES',
 'FrodoKEM-976-SHAKE', 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE']
Enabled signature mechanisms:
['Dilithium2', 'Dilithium3', 'Dilithium5', 'ML-DSA-44-ipd', 'ML-DSA-44',
 'ML-DSA-65-ipd', 'ML-DSA-65', 'ML-DSA-87-ipd', 'ML-DSA-87', 'Falcon-512',
 'Falcon-1024', 'Falcon-padded-512', 'Falcon-padded-1024',
 'SPHINCS+-SHA2-128f-simple', 'SPHINCS+-SHA2-128s-simple',
 'SPHINCS+-SHA2-192f-simple', 'SPHINCS+-SHA2-192s-simple',
 'SPHINC

In [1]:
import oqs
from pprint import pprint
from timeit import default_timer

#print("Enabled signature mechanisms:")
#sigs = oqs.get_enabled_sig_mechanisms()
#pprint(sigs, compact=True)

message = "This is the message to sign".encode()

# Create signer and verifier with sample signature mechanisms
#sigalg = "Dilithium2"

ML_DSA_alg = ['ML-DSA-44',
              'ML-DSA-65',
              'ML-DSA-87']

def ML_DSA(message: str, ML_DSA_alg: str) -> dict:
    """Function that uses the ML-DSA to sign and verify a message"""
    with oqs.Signature(ML_DSA_alg) as signer:
        with oqs.Signature(ML_DSA_alg) as verifier:
            #print("\nSignature details:")
            #pprint(signer.details)
            
            # Dictionary to save timestamps
            ML_DSA_time = {
                "key_pair_generation": [],
                "sign_message": [],
                "verify_signature": []
                }

            # Signer generates its keypair
            ML_DSA_time["key_pair_generation"].append(default_timer())
            signer_public_key = signer.generate_keypair()
            # Optionally, the secret key can be obtained by calling export_secret_key()
            # and the signer can later be re-instantiated with the key pair:
            # secret_key = signer.export_secret_key()

            # Store key pair, wait... (session resumption):
            # signer = oqs.Signature(sigalg, secret_key)
            ML_DSA_time["key_pair_generation"].append(default_timer())

            # Signer signs the message
            ML_DSA_time["sign_message"].append(default_timer())
            signature = signer.sign(message)
            ML_DSA_time["sign_message"].append(default_timer())

            # Verifier verifies the signature
            ML_DSA_time["verify_signature"].append(default_timer())
            is_valid = verifier.verify(message, signature, signer_public_key)
            ML_DSA_time["verify_signature"].append(default_timer())

            #print("\nValid signature?", is_valid)
            
            return ML_DSA_time

        
average_time = {
    "key_pair_generation_ML-DSA-44": [],
    "sign_message_ML-DSA-44": [],
    "verify_signature_ML-DSA-44": [],
    "total_ML-DSA-44": [],
    
    "key_pair_generation_ML-DSA-65": [],
    "sign_message_ML-DSA-65": [],
    "verify_signature_ML-DSA-65": [],
    "total_ML-DSA-65": [],
    
    "key_pair_generation_ML-DSA-87": [],
    "sign_message_ML-DSA-87": [],
    "verify_signature_ML-DSA-87": [],
    "total_ML-DSA-87": [],
    }
        
        
# We repeat each algorithm a hundred times
for _ in range(1000):
    #for algorithm in ["ML-KEM-512", "ML-KEM-768", "ML-KEM-1024"]:
    for algorithm in ML_DSA_alg:
        times = ML_DSA(message, algorithm)
        #print(times)
        total_time = 0
        for key, value in times.items():
            #print(key)
            #print(value)
            total_time += value[1] - value[0]
            average_time[key+"_"+algorithm].append(value[1] - value[0])
            #print(average_time[key+"_"+algorithm])
        average_time["total_"+algorithm].append(total_time)

        
# We show the final results
for key, value in average_time.items():
    if "key_pair_generation" in key: print(f"\t ### {key[20:]} ###")
    print(f"{key}")
    print(sum(value))

	 ### ML-DSA-44 ###
key_pair_generation_ML-DSA-44
0.5246749999423628
sign_message_ML-DSA-44
1.7664715999926557
verify_signature_ML-DSA-44
0.4828662997315405
total_ML-DSA-44
2.774012899666559
	 ### ML-DSA-65 ###
key_pair_generation_ML-DSA-65
0.8444055998625117
sign_message_ML-DSA-65
2.744117699949129
verify_signature_ML-DSA-65
0.7839647999571753
total_ML-DSA-65
4.372488099768816
	 ### ML-DSA-87 ###
key_pair_generation_ML-DSA-87
1.3083038000913803
sign_message_ML-DSA-87
3.4595055001045694
verify_signature_ML-DSA-87
1.288624899876595
total_ML-DSA-87
6.056434200072545


## SLH-DSA Scheme

In [2]:
!pip install pyspx

  Using cached PySPX-0.5.0.tar.gz (454 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for cffi>=1.0.0 from https://files.pythonhosted.org/packages/5a/c7/694814b3757878b29da39bc2f0cf9d20295f4c1e0a0bde7971708d5f23f8/cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata
  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata (1.5 kB)
  Obtaining dependency information for pycparser from https://files.pythonhosted.org/packages/13/a3/a812df4e2dd5696d1f351d58b8fe16a405b234ad2886a0dab9183fb78109/pycparser-2.22-py3-none-any.whl.metadata
  Using cached pycparser-2.22-py3-none

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [79 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\bindings.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\build.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_128f.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_128s.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_192f.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_192s.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_256f.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\haraka_256s.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\sha2_128f.py -> build\lib.win-amd64-cpython-311\pyspx
  copying src\pyspx\sha2_128s.py -> build\lib.win-amd64-cpython-311\pyspx
  cop

In [ ]:
from pyspx import shake_128f
import os, binascii

def slh_shake_128_Fast():
  # Key generation: private + public key
  seed = os.urandom(shake_128f.crypto_sign_SEEDBYTES)
  public_key, secret_key = shake_128f.generate_keypair(seed)

  # Sign message and verify signature
  message = b'Message for SPHINCS+ shake256_128f signing'
  signature = shake_128f.sign(message, secret_key)
  valid = shake_128f.verify(message, signature, public_key)

Public key: b'94b72a6426b4baa036c4527270d564e425c0f05b2b025f02202014b6e0581cc9'
Private key: b'830e8ab337e4c4f17fb44c80a60767e6f4be2e9c585d8258d069b159258c0abe94b72a6426b4baa036c4527270d564e425c0f05b2b025f02202014b6e0581cc9'
Message: b'Message for SPHINCS+ shake256_128f signing'
Signature: b'...'
Signature valid? True
Tampered message: b'Tampered msg'
Tampered signature valid? False


In [ ]:
from pyspx import shake_128f,shake_192f,shake_256f
from pyspx import sha2_128f,sha2_192f,sha2_256f
from timeit import default_timer
import os
import importlib

paramsets = [
    'shake_128f',
    'shake_192f',
    'shake_256f',
    'sha2_128f',
    'sha2_192f',
    'sha2_256f',
]

instances = []

for paramset in paramsets:
    instances.append(importlib.import_module('pyspx.' + paramset))

average_time = {
    "key_pair_generation_shake_128f": [],
    "sign_message_shake_128f": [],
    "verify_signature_shake_128f": [],
    "total_shake_128f": [],
    "key_pair_generation_shake_192f": [],
    "sign_message_shake_192f": [],
    "verify_signature_shake_192f": [],
    "total_shake_192f": [],
    "key_pair_generation_shake_256f": [],
    "sign_message_shake_256f": [],
    "verify_signature_shake_256f": [],
    "total_shake_256f": [],

    "key_pair_generation_sha2_128f": [],
    "sign_message_sha2_128f": [],
    "verify_signature_sha2_128f": [],
    "total_sha2_128f": [],
    "key_pair_generation_sha2_192f": [],
    "sign_message_sha2_192f": [],
    "verify_signature_sha2_192f": [],
    "total_sha2_192f": [],
    "key_pair_generation_sha2_256f": [],
    "sign_message_sha2_256f": [],
    "verify_signature_sha2_256f": [],
    "total_sha2_256f": [],
    }

def SLH_DSA(message, SLH_DSA_alg):
  """Function that uses the SLH_DSA scheme to generate a pair of keys,
  sign a message and verify the signature"""
  # Dictionary to save timestamps
  SLH_DSA_time = {
    "key_pair_generation": [],
    "sign_message": [],
    "verify_signature": []
  }

  # Key generation: private + public key
  SLH_DSA_time["key_pair_generation"].append(default_timer())
  seed = os.urandom(SLH_DSA_alg.crypto_sign_SEEDBYTES)
  public_key, secret_key = SLH_DSA_alg.generate_keypair(seed)
  SLH_DSA_time["key_pair_generation"].append(default_timer())

  # Sign message
  SLH_DSA_time["sign_message"].append(default_timer())
  signature = SLH_DSA_alg.sign(message, secret_key)
  SLH_DSA_time["sign_message"].append(default_timer())

  # Verify signature
  SLH_DSA_time["verify_signature"].append(default_timer())
  valid = SLH_DSA_alg.verify(message, signature, public_key)
  SLH_DSA_time["verify_signature"].append(default_timer())

  return SLH_DSA_time

# We repeat each algorithm a hundred times
message = b'Message for SPHINCS+ signing'
for _ in range(1000):
    apunt_algorithm = 0
    for algorithm in instances:
        times = SLH_DSA(message, algorithm)
        total_time = 0
        for key, value in times.items():
            total_time += value[1] - value[0]
            average_time[key+"_"+paramsets[apunt_algorithm]].append(value[1] - value[0])
        average_time["total_"+paramsets[apunt_algorithm]].append(total_time)

        if (apunt_algorithm == 5):
          apunt_algorithm = 0
        else:
          apunt_algorithm += 1


# We show the final results
for key, value in average_time.items():
    if "key_pair_generation" in key: print(f"\t ### {key[20:]} ###")
    print(f"{key}")
    print(sum(value))


	 ### shake_128f ###
key_pair_generation_shake_128f
7.501941970997905
sign_message_shake_128f
172.26452088502901
verify_signature_shake_128f
10.684129232002306
total_shake_128f
190.45059208802923
	 ### shake_192f ###
key_pair_generation_shake_192f
10.99226509100663
sign_message_shake_192f
276.449760770005
verify_signature_shake_192f
15.729601382976398
total_shake_192f
303.17162724398804
	 ### shake_256f ###
key_pair_generation_shake_256f
28.96065935698607
sign_message_shake_256f
569.2502128249935
verify_signature_shake_256f
16.120847206007966
total_shake_256f
614.3317193879875
	 ### sha2_128f ###
key_pair_generation_sha2_128f
4.595611365009972
sign_message_sha2_128f
105.22615454800234
verify_signature_sha2_128f
6.651644311978998
total_sha2_128f
116.47341022499131
	 ### sha2_192f ###
key_pair_generation_sha2_192f
6.6841801739929
sign_message_sha2_192f
169.5454105529725
verify_signature_sha2_192f
9.780191888004993
total_sha2_192f
186.0097826149704
	 ### sha2_256f ###
key_pair_generation_